<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/cdsoption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper for formatting (optional, but can make messages clearer)
def format_value(v):
    return f"{v:.6f}"

# Test Suite for CDS Options
class CdsOptionTests(unittest.TestCase):

    def setUp(self):
        """Set up the calculation date and other common parameters if needed."""
        self.original_eval_date = ql.Settings.instance().evaluationDate
        # Set a fixed date for the test consistent with C++
        self.cachedToday = ql.Date(10, ql.December, 2007)
        ql.Settings.instance().evaluationDate = self.cachedToday

        # Common objects (can also be defined within the test method)
        self.calendar = ql.TARGET()
        self.dayCounter = ql.Actual360() # Day counter for hazard rate and risk-free rate
        self.convention = ql.ModifiedFollowing # Business day convention for schedule

        # Risk-free rate term structure
        self.riskFreeRate = 0.02
        self.riskFree = ql.RelinkableYieldTermStructureHandle()
        self.riskFree.linkTo(ql.FlatForward(self.cachedToday, self.riskFreeRate, self.dayCounter))

        # Default probability term structure
        self.hazardRateQuote = ql.SimpleQuote(0.001)
        self.hazardRateHandle = ql.QuoteHandle(self.hazardRateQuote)
        self.defaultProbability = ql.DefaultProbabilityTermStructureHandle(
            ql.FlatHazardRate(0, self.calendar, self.hazardRateHandle, self.dayCounter)
        )

        # CDS schedule parameters
        self.expiry = self.calendar.advance(self.cachedToday, 9, ql.Months)
        self.startDate = self.calendar.advance(self.expiry, 1, ql.Months)
        self.maturity = self.calendar.advance(self.startDate, 7, ql.Years)
        self.cdsSchedule = ql.Schedule(self.startDate, self.maturity, ql.Period(ql.Quarterly),
                                      self.calendar, self.convention, self.convention,
                                      ql.DateGeneration.Forward, False)

        # CDS parameters
        self.notional = 1000000.0
        self.recoveryRate = 0.4
        self.cdsDayCounter = ql.Actual360() # Day counter for CDS premium leg (can differ from others)

        # CDS pricing engine
        self.swapEngine = ql.MidPointCdsEngine(self.defaultProbability, self.recoveryRate, self.riskFree)

        # CDS option parameters
        self.cdsVolQuote = ql.SimpleQuote(0.20)
        self.cdsVolHandle = ql.QuoteHandle(self.cdsVolQuote)
        self.exercise = ql.EuropeanExercise(self.expiry)

        # CDS option pricing engine
        self.optionEngine = ql.BlackCdsOptionEngine(self.defaultProbability, self.recoveryRate,
                                                   self.riskFree, self.cdsVolHandle)


    def tearDown(self):
        """Restore the calculation date."""
        ql.Settings.instance().evaluationDate = self.original_eval_date


    def testCached(self):
        """Testing CDS-option value against cached values."""
        print("Testing CDS-option value against cached values...")

        # --- Calculate fair strike for the underlying CDS ---
        # Need a dummy swap to calculate fair spread
        # The premium rate (0.001 in C++ test) is used to define the swap *object*,
        # but the fair spread is needed for the option strike.
        dummy_swap_for_fair_spread = ql.CreditDefaultSwap(
            ql.Protection.Seller, self.notional, 0.0, # Use 0 spread initially
            self.cdsSchedule, self.convention, self.cdsDayCounter)
        dummy_swap_for_fair_spread.setPricingEngine(self.swapEngine)

        fair_strike = dummy_swap_for_fair_spread.fairSpread()
        # print(f"Calculated fair strike: {fair_strike}") # For debugging

        # --- Create underlying CDS for the option using the fair strike ---
        underlying_seller = ql.CreditDefaultSwap(
            ql.Protection.Seller, self.notional, fair_strike, self.cdsSchedule,
            self.convention, self.cdsDayCounter)
        underlying_seller.setPricingEngine(self.swapEngine) # Engine for underlying value if needed

        # --- Create and price the CDS option (payer perspective - buying protection) ---
        option1 = ql.CdsOption(underlying_seller, self.exercise)
        option1.setPricingEngine(self.optionEngine)

        cachedValue1 = 270.976348
        calculatedValue1 = option1.NPV()

        self.assertAlmostEqual(calculatedValue1, cachedValue1, delta=1.0e-5,
                               msg=(f"Failed to reproduce cached value (Seller protection underlying):\n"
                                    f"    calculated: {format_value(calculatedValue1)}\n"
                                    f"    expected:   {format_value(cachedValue1)}"))

        # --- Create underlying CDS for the option (buyer perspective - selling protection) ---
        underlying_buyer = ql.CreditDefaultSwap(
            ql.Protection.Buyer, self.notional, fair_strike, self.cdsSchedule,
            self.convention, self.cdsDayCounter)
        underlying_buyer.setPricingEngine(self.swapEngine)

        # --- Create and price the CDS option (receiver perspective - selling protection) ---
        option2 = ql.CdsOption(underlying_buyer, self.exercise)
        option2.setPricingEngine(self.optionEngine)

        # The NPV of an option to buy protection should equal the NPV of an option to sell protection
        # when struck at the forward fair spread (assuming Black model symmetry).
        cachedValue2 = 270.976348
        calculatedValue2 = option2.NPV()

        self.assertAlmostEqual(calculatedValue2, cachedValue2, delta=1.0e-5,
                               msg=(f"Failed to reproduce cached value (Buyer protection underlying):\n"
                                    f"    calculated: {format_value(calculatedValue2)}\n"
                                    f"    expected:   {format_value(cachedValue2)}"))

if __name__ == '__main__':
    print("Presolve testQuantLib.py ...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)